In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from collections import Counter
import pandas as pd

# Load Dataset 

In [ ]:
dataset = pd.read_parquet(r'../local_dataset/dataset/merged_full.parquet')
dataset.drop(columns=['lon', 'lat', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4'], inplace=True, errors='ignore')
print("=== Dataset Types :\n", dataset.dtypes)

dataset_numeric_columns = dataset.select_dtypes(include=['number']).columns
dataset_string_columns = dataset.select_dtypes(include=['object']).columns
dataset[dataset_numeric_columns] = dataset[dataset_numeric_columns].fillna(dataset[dataset_numeric_columns].mean())
dataset[dataset_string_columns] = dataset[dataset_string_columns].fillna(dataset[dataset_string_columns].mode().iloc[0])

print("=== Null Values :\n", dataset.isnull().sum())

X = dataset.drop('fire', axis=1)
X_numeric_columns = X.select_dtypes(include=['number']).columns
X[X_numeric_columns] = StandardScaler().fit_transform(X[X_numeric_columns])
X = pd.get_dummies(X, columns=X.select_dtypes(include=['object']).columns)

Y = dataset[['fire']]

print("=== X columns:", X.columns)
print("=== Y index:", Y.index)

X_train, X_test, Y_train, Y_test = train_test_split(
    X.to_numpy(), Y.to_numpy(),
    test_size=0.2,      # 20% test
    random_state=42,    # for reproducibility
    shuffle=True        # default True
)

--- Dataset Types :
 fire                        int64
tmax_s1                   float64
tmax_s2                   float64
tmax_s3                   float64
tmax_s4                   float64
amplitude_thermique_s1    float64
amplitude_thermique_s2    float64
amplitude_thermique_s3    float64
amplitude_thermique_s4    float64
GRIDCODE                  float64
log_area_sqm              float64
lcc_code_encoded          float64
elevation                 float32
COARSE                    float64
SAND                      float64
SILT                      float64
CLAY                      float64
TEXTURE_USDA              float64
TEXTURE_SOTER              object
BULK                      float64
REF_BULK                  float64
ORG_CARBON                float64
PH_WATER                  float64
TOTAL_N                   float64
CN_RATIO                  float64
CEC_SOIL                  float64
CEC_CLAY                  float64
CEC_EFF                   float64
TEB                       f

# SKlearn Model Training

In [3]:
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

knn.fit(X_train, Y_train)


,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,-1


# Scratch Model Training

In [7]:

import numpy as np
from collections import Counter

class FastKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _euclidean_distances(self, X):
        """
        Vectorized distance computation:
        Computes distances from many samples X to all training samples.
        Uses (a - b)^2 = a^2 + b^2 - 2ab trick.
        """

        # X: (n_test, n_features)
        # X_train: (n_train, n_features)

        X_square = np.sum(X**2, axis=1).reshape(-1, 1)          # (n_test, 1)
        train_square = np.sum(self.X_train**2, axis=1)          # (n_train,)
        cross = X @ self.X_train.T                               # (n_test, n_train)

        # squared distances (no sqrt needed)
        dists = X_square + train_square - 2 * cross
        return dists

    def predict(self, X):
        # Compute all distances in one shot
        dists = self._euclidean_distances(X)

        # Indices of k nearest neighbors per test sample
        k_indices = np.argpartition(dists, self.k, axis=1)[:, :self.k]

        # Convert labels
        k_labels = self.y_train[k_indices]

        # Majority vote across axis 1
        preds = np.array([Counter(row).most_common(1)[0][0] for row in k_labels])
        return preds



knn_scratch = FastKNN(k=5)
knn_scratch.fit(X_train, Y_train)

# SKlearn Model Testing

In [22]:
predictions = knn.predict(X_test)
accuracy = (predictions == Y_test).mean()
print(f'KNN Model Accuracy: {accuracy * 100:.2f}%')

KNN Model Accuracy: 88.98%


# Scratch Model Testing

In [ ]:
# predictions = knn_scratch.predict(X_test)
# accuracy = (predictions == Y_test).mean() 
# print(f'Scratch KNN Model Accuracy: {accuracy * 100:.2f}%')